In [1]:
import os
import json
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
from collections import defaultdict, OrderedDict
import shutil
from tqdm import tqdm

### Json에 있는 Segmentation Annotation 추출 및 line coord 교체

In [6]:
data = pd.read_csv("/opt/ml/team_github/utils/line_crosswalk_(12_14)_filtered_1.csv")
data

,file_locate,file_name,class,coord,coord_num,ratio
0,[원천]1920_1080_night_train_d_1.zip,14250087.jpg,crosswalk,"[[4, 845], [1762, 845], [1836, 847], [1910, 96...",16,20.56
1,[원천]1920_1080_night_train_d_1.zip,14422355.jpg,crosswalk,"[[57, 852], [1074, 697], [1640, 613], [1901, 5...",18,29.05
2,[원천]1920_1080_night_train_d_1.zip,14641550.jpg,crosswalk,"[[986, 635], [748, 664], [11, 757], [1, 762], ...",22,23.19
3,[원천]1920_1080_night_train_d_1.zip,14641634.jpg,crosswalk,"[[0, 1072], [1, 1072], [1819, 1072], [1851, 88...",26,29.41
4,[원천]1920_1080_night_train_d_1.zip,14641689.jpg,crosswalk,"[[370, 873], [754, 849], [1867, 788], [1873, 1...",14,20.60
...,...,...,...,...,...,...
3005,[원천]d_1920_1080_daylight_train_9.zip,14768626.jpg,crosswalk,"[[2, 747], [588, 750], [1155, 742], [1352, 724...",14,29.80
3006,[원천]d_1920_1080_daylight_train_9.zip,14768631.jpg,crosswalk,"[[5, 692], [406, 694], [838, 695], [1152, 694]...",20,25.95
3007,[원천]d_1920_1080_daylight_train_9.zip,14769815.jpg,crosswalk,"[[5, 880], [755, 876], [1786, 861], [1918, 852...",18,20.62
3008,[원천]d_1920_1080_daylight_train_9.zip,14772642.jpg,crosswalk,"[[2, 913], [2, 1079], [567, 1076], [1914, 468]...",14,23.91


In [3]:
data[data['file_locate'].str.contains("c_1280_720")]

,file_locate,file_name,class,coord
4222,[원천]c_1280_720_daylight_train_1.zip,16639018.jpg,crosswalk,"[[53, 356], [61, 365], [767, 361], [845, 350],..."
4223,[원천]c_1280_720_daylight_train_1.zip,16685438.jpg,crosswalk,"[[1064, 353], [1274, 398], [1268, 473], [776, ..."
4224,[원천]c_1280_720_daylight_train_1.zip,16685438.jpg,crosswalk,"[[344, 372], [244, 454], [159, 445], [156, 385]]"
4225,[원천]c_1280_720_daylight_train_1.zip,6938831.jpg,crosswalk,"[[396, 334], [427, 334], [453, 336], [466, 336..."
4226,[원천]c_1280_720_daylight_train_1.zip,6938831.jpg,crosswalk,"[[132, 424], [225, 397], [324, 405], [416, 407..."
...,...,...,...,...
8365,[원천]c_1280_720_daylight_train_1.zip,16898816.jpg,crosswalk,"[[45, 472], [196, 503], [387, 526], [834, 548]..."
8366,[원천]c_1280_720_daylight_train_1.zip,16619970.jpg,crosswalk,"[[0, 541], [0, 611], [110, 610], [221, 622], [..."
8367,[원천]c_1280_720_daylight_train_1.zip,16923208.jpg,crosswalk,"[[87, 477], [459, 453], [575, 444], [579, 459]..."
8368,[원천]c_1280_720_daylight_train_1.zip,16705931.jpg,crosswalk,"[[803, 544], [624, 543], [634, 565], [858, 633..."


In [4]:
data.drop(data[data['file_locate'].str.contains("c_1280_720")].index, inplace=True)
data.reset_index(drop=True, inplace=True)
data

,file_locate,file_name,class,coord
0,[원천]1920_1080_night_train_d_1.zip,15691903.jpg,crosswalk,"[[1112, 708], [628, 765], [330, 788], [3, 820]..."
1,[원천]1920_1080_night_train_d_1.zip,15484144.jpg,crosswalk,"[[118, 806], [1644, 822], [1914, 817], [1920, ..."
2,[원천]1920_1080_night_train_d_1.zip,15484144.jpg,crosswalk,"[[2, 636], [430, 623], [791, 615], [1343, 598]..."
3,[원천]1920_1080_night_train_d_1.zip,14427380.jpg,crosswalk,"[[6, 639], [552, 587], [378, 572], [7, 578], [..."
4,[원천]1920_1080_night_train_d_1.zip,14427380.jpg,crosswalk,"[[937, 602], [1219, 624], [1579, 653], [1527, ..."
...,...,...,...,...
179412,[원천]d_1920_1080_daylight_train_9.zip,14548161.jpg,crosswalk,"[[522, 612], [901, 601], [1074, 598], [1149, 6..."
179413,[원천]d_1920_1080_daylight_train_9.zip,14548161.jpg,crosswalk,"[[1318, 607], [1716, 601], [1716, 666], [1575,..."
179414,[원천]d_1920_1080_daylight_train_9.zip,14758492.jpg,crosswalk,"[[30, 635], [658, 563], [617, 558], [78, 619]]"
179415,[원천]d_1920_1080_daylight_train_9.zip,14758492.jpg,crosswalk,"[[1038, 520], [1276, 519], [1913, 515], [1917,..."


In [5]:
print(len(set(list(data['file_locate'].values))))
print(len(set(list(data['file_name'].values))))

17
108426


In [15]:
coord_num = []

for each_data in data['coord'].values:
    coord_num.append(len(each_data.split(",")))

In [16]:
data['coord_num'] = coord_num
data

,file_locate,file_name,class,coord,coord_num
0,[원천]1920_1080_night_train_d_1.zip,15691903.jpg,crosswalk,"[[1112, 708], [628, 765], [330, 788], [3, 820]...",18
1,[원천]1920_1080_night_train_d_1.zip,15484144.jpg,crosswalk,"[[118, 806], [1644, 822], [1914, 817], [1920, ...",14
2,[원천]1920_1080_night_train_d_1.zip,15484144.jpg,crosswalk,"[[2, 636], [430, 623], [791, 615], [1343, 598]...",18
3,[원천]1920_1080_night_train_d_1.zip,14427380.jpg,crosswalk,"[[6, 639], [552, 587], [378, 572], [7, 578], [...",12
4,[원천]1920_1080_night_train_d_1.zip,14427380.jpg,crosswalk,"[[937, 602], [1219, 624], [1579, 653], [1527, ...",12
...,...,...,...,...,...
179412,[원천]d_1920_1080_daylight_train_9.zip,14548161.jpg,crosswalk,"[[522, 612], [901, 601], [1074, 598], [1149, 6...",14
179413,[원천]d_1920_1080_daylight_train_9.zip,14548161.jpg,crosswalk,"[[1318, 607], [1716, 601], [1716, 666], [1575,...",8
179414,[원천]d_1920_1080_daylight_train_9.zip,14758492.jpg,crosswalk,"[[30, 635], [658, 563], [617, 558], [78, 619]]",8
179415,[원천]d_1920_1080_daylight_train_9.zip,14758492.jpg,crosswalk,"[[1038, 520], [1276, 519], [1913, 515], [1917,...",12


In [20]:
data.drop(data[data['coord_num'] < 6].index, inplace=True)
data.reset_index(drop=True, inplace=True)
data

,file_locate,file_name,class,coord,coord_num
0,[원천]1920_1080_night_train_d_1.zip,15691903.jpg,crosswalk,"[[1112, 708], [628, 765], [330, 788], [3, 820]...",18
1,[원천]1920_1080_night_train_d_1.zip,15484144.jpg,crosswalk,"[[118, 806], [1644, 822], [1914, 817], [1920, ...",14
2,[원천]1920_1080_night_train_d_1.zip,15484144.jpg,crosswalk,"[[2, 636], [430, 623], [791, 615], [1343, 598]...",18
3,[원천]1920_1080_night_train_d_1.zip,14427380.jpg,crosswalk,"[[6, 639], [552, 587], [378, 572], [7, 578], [...",12
4,[원천]1920_1080_night_train_d_1.zip,14427380.jpg,crosswalk,"[[937, 602], [1219, 624], [1579, 653], [1527, ...",12
...,...,...,...,...,...
179275,[원천]d_1920_1080_daylight_train_9.zip,14548161.jpg,crosswalk,"[[522, 612], [901, 601], [1074, 598], [1149, 6...",14
179276,[원천]d_1920_1080_daylight_train_9.zip,14548161.jpg,crosswalk,"[[1318, 607], [1716, 601], [1716, 666], [1575,...",8
179277,[원천]d_1920_1080_daylight_train_9.zip,14758492.jpg,crosswalk,"[[30, 635], [658, 563], [617, 558], [78, 619]]",8
179278,[원천]d_1920_1080_daylight_train_9.zip,14758492.jpg,crosswalk,"[[1038, 520], [1276, 519], [1913, 515], [1917,...",12


In [49]:
data['size'] = None
data['ratio'] = None
data

,file_locate,file_name,class,coord,coord_num,size,ratio
0,[원천]1920_1080_night_train_d_1.zip,15691903.jpg,crosswalk,"[[1112, 708], [628, 765], [330, 788], [3, 820]...",18,None,None
1,[원천]1920_1080_night_train_d_1.zip,15484144.jpg,crosswalk,"[[118, 806], [1644, 822], [1914, 817], [1920, ...",14,None,None
2,[원천]1920_1080_night_train_d_1.zip,15484144.jpg,crosswalk,"[[2, 636], [430, 623], [791, 615], [1343, 598]...",18,None,None
3,[원천]1920_1080_night_train_d_1.zip,14427380.jpg,crosswalk,"[[6, 639], [552, 587], [378, 572], [7, 578], [...",12,None,None
4,[원천]1920_1080_night_train_d_1.zip,14427380.jpg,crosswalk,"[[937, 602], [1219, 624], [1579, 653], [1527, ...",12,None,None
...,...,...,...,...,...,...,...
179275,[원천]d_1920_1080_daylight_train_9.zip,14548161.jpg,crosswalk,"[[522, 612], [901, 601], [1074, 598], [1149, 6...",14,None,None
179276,[원천]d_1920_1080_daylight_train_9.zip,14548161.jpg,crosswalk,"[[1318, 607], [1716, 601], [1716, 666], [1575,...",8,None,None
179277,[원천]d_1920_1080_daylight_train_9.zip,14758492.jpg,crosswalk,"[[30, 635], [658, 563], [617, 558], [78, 619]]",8,None,None
179278,[원천]d_1920_1080_daylight_train_9.zip,14758492.jpg,crosswalk,"[[1038, 520], [1276, 519], [1913, 515], [1917,...",12,None,None


In [51]:
len(data)

179280

In [78]:
def calculate_area(coord_list):
    dummy = np.zeros((1080,1920))
    dummy = cv2.fillPoly(dummy, [np.array(coord_list)], 255)
    _, counts = np.where(dummy==255)
    return round(len(counts)*100/(1920*1080),2)

In [73]:
import ast

In [80]:
area_datas = []
for each_coord in tqdm(data.to_dict("record")):
    area_datas.append(calculate_area(ast.literal_eval(each_coord['coord'])))

/opt/conda/envs/final/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  
100%|██████████| 179280/179280 [32:49<00:00, 91.02it/s] 


In [83]:
data['ratio'] = area_datas
data.to_csv('new_seg_dataset.csv', encoding='utf-8')

In [107]:
new_seg = {}

print(len(data["coord"]))

for idx, each_record in tqdm(enumerate(data.to_dict("record"))):
    # seg_datas = ast.literal_eval(each_record["coord"])
    seg_datas = each_record["coord"]
    h = 1080
    w = 1920

    coord = []
    for index, target in enumerate(seg_datas):
        for seg_idx in range(0, len(target), 2):
            x, y = target[seg_idx], target[seg_idx+1]
            if x < 0:
                x = 0
            if x >= w:
                x = w-1
            if y < 0:
                y = 0
            if y >= h:
                y = h-1
            coord.append([x, y])
        new_seg.update({idx: coord})

179280


/opt/conda/envs/final/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  """
179280it [00:02, 62268.17it/s]


In [108]:
data = data.drop('coord', axis=1)
s_new_seg = pd.Series(new_seg)
data.insert(3, "coord", s_new_seg) # insert new_series
data

,file_locate,file_name,class,coord,coord_num,ratio
0,[원천]1920_1080_night_train_d_1.zip,15691903.jpg,crosswalk,"[[1112, 708], [628, 765], [330, 788], [3, 820]...",18,3.93
1,[원천]1920_1080_night_train_d_1.zip,15484144.jpg,crosswalk,"[[118, 806], [1644, 822], [1914, 817], [1919, ...",14,24.37
2,[원천]1920_1080_night_train_d_1.zip,15484144.jpg,crosswalk,"[[2, 636], [430, 623], [791, 615], [1343, 598]...",18,1.30
3,[원천]1920_1080_night_train_d_1.zip,14427380.jpg,crosswalk,"[[6, 639], [552, 587], [378, 572], [7, 578], [...",12,1.01
4,[원천]1920_1080_night_train_d_1.zip,14427380.jpg,crosswalk,"[[937, 602], [1219, 624], [1579, 653], [1527, ...",12,0.53
...,...,...,...,...,...,...
179275,[원천]d_1920_1080_daylight_train_9.zip,14548161.jpg,crosswalk,"[[522, 612], [901, 601], [1074, 598], [1149, 6...",14,0.45
179276,[원천]d_1920_1080_daylight_train_9.zip,14548161.jpg,crosswalk,"[[1318, 607], [1716, 601], [1716, 666], [1575,...",8,1.05
179277,[원천]d_1920_1080_daylight_train_9.zip,14758492.jpg,crosswalk,"[[30, 635], [658, 563], [617, 558], [78, 619]]",8,0.31
179278,[원천]d_1920_1080_daylight_train_9.zip,14758492.jpg,crosswalk,"[[1038, 520], [1276, 519], [1913, 515], [1917,...",12,1.45


In [109]:
data.to_csv('new_seg_dataset_1.csv', encoding='utf-8')